# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [345]:
from ast import Lambda
from ctypes.wintypes import FLOAT
from matplotlib.axis import Axis
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_data_nhl():
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    #get 2018 data
    nhl_df = nhl_df.loc[nhl_df['year'] == 2018]

    #Remove division header
    nhl_df.drop([0,9,18,26], axis=0, inplace=True)

    # get relevant columns
    nhl_df = nhl_df.filter(['team', 'W', 'L', 'year'])

    #Clean team names
    nhl_df['team'] = nhl_df['team'].apply(lambda x: re.sub(r"\*", "", x))

    cities['NHL'] = cities['NHL'].apply(lambda x: re.sub(r"\[.*\]", "",x))

    # Seprarate the name of the different teams merged together
    cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers Islanders Devils",
                                        "KingsDucks": "Kings Ducks",
                                        "Blue Jackets": "Blue-Jackets",
                                        "Maple Leafs": "Maple-Leafs",
                                        "Golden Knights": "Golden-Knights",
                                        "Red Wings": "Red-Wings"
                                                        })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(" "))
    cities = cities.explode("NHL")
    cities = cities.filter(['Metropolitan area', 'Population (2016 est.)[8]', 'NHL'])



    nhl_df["team"] = nhl_df["team"].replace({"Toronto Maple Leafs": "Toronto Maple-Leafs",
                                        "Detroit Red Wings": "Detroit Red-Wings",
                                        "Columbus Blue Jackets": "Columbus Blue-Jackets",
                                        "Vegas Golden Knights": "Vegas Golden-Knights"
                                                        })
    
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(" ")[-1])

    df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
    df.drop(['team', 'NHL', 'year'], axis=1, inplace=True)

    df['W/L%'] = df['W'].astype(float) / (df['W'].astype(float) + df['L'].astype(float))
   
    df2 = df.groupby(['Metropolitan area']).mean()
    df1 = df.drop_duplicates(['Metropolitan area']).reset_index(drop=True)


    df1.drop('W/L%', axis=1 , inplace = True)
    df3 = pd.merge(df2, df1, on='Metropolitan area')

    df3 = df3.filter(['Metropolitan area', 'Population (2016 est.)[8]', 'W/L%' ])

    return df3
    

def nhl_correlation(): 
    # YOUR CODE HERE    
    #raise NotImplementedError()
    df = clean_data_nhl()
    population_by_region = df["Population (2016 est.)[8]"].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df["W/L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)

nhl_correlation()

(0.012486162921209923, 0.9497182859911781)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [287]:
from posixpath import split
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clean_data_nba():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    nba_df =nba_df.loc[nba_df['year'] == 2018]

    # get relevant columns
    nba_df = nba_df.filter(['team', 'W/L%'])

    nba_df['team'] = nba_df['team'].apply(lambda x: re.sub(r"\*?\s\(\d*\)", "",x))

    nba_df['team'] = nba_df['team'].replace({'Portland Trail Blazers' : 'Portland Trail-Blazers'})

    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(" ")[-1])


    
    
    cities = cities.filter(['Metropolitan area', 'Population (2016 est.)[8]', 'NBA'])

    cities['NBA'] = cities['NBA'].apply(lambda x: re.sub(r"\[.*\]", "", x))
    
    cities['NBA'] = cities['NBA'].replace({ 
                                            'KnicksNets' : 'Knicks Nets',
                                            'LakersClippers' : 'Lakers Clippers',
                                            'Trail Blazers' : 'Trail-Blazers'
    })

    cities['NBA'] = cities['NBA'].apply(lambda x: x.split(" "))
    cities = cities.explode("NBA")


    df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")

    df.drop(['team', 'NBA'], axis= 1, inplace=True)
    
    df['W/L%'] = df['W/L%'].astype(float)
    
    df2 = df.groupby(['Metropolitan area']).mean()
    df1 = df.drop_duplicates(['Metropolitan area']).reset_index(drop=True)
    df1.drop('W/L%', axis=1 , inplace = True)


    df3 = pd.merge(df1 , df2 , on= 'Metropolitan area')

    return df3
    


def nba_correlation():
    #YOUR CODE HERE
    #    raise NotImplementedError()
    df = clean_data_nba()
        
    population_by_region = df["Population (2016 est.)[8]"].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df["W/L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)
    
nba_correlation()


(-0.17636350642182935, 0.36932106185547353)

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [342]:
from matplotlib.pyplot import axis
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clean_data_mlb():
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]


    mlb_df = mlb_df.loc[mlb_df['year']== 2018]

    mlb_df['team'] = mlb_df['team'].replace({
                                            'Boston Red Sox' : 'Boston Red-Sox',
                                            'Toronto Blue Jays' : 'Toronto Blue-Jays',
                                            'Chicago White Sox' : 'Chicago White-Sox'
    })

    mlb_df = mlb_df.filter(['team', 'W-L%'])

    mlb_df['team'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])

    cities = cities.filter(['Metropolitan area', 'Population (2016 est.)[8]', 'MLB'])

    cities['MLB'] = cities['MLB'].apply(lambda x: re.sub(r'\[.*\]',"", x))

    cities['MLB'] = cities['MLB'].replace({
                                            'YankeesMets' : 'Yankees Mets',
                                            'DodgersAngels' : 'Dodgers Angels', 
                                            'GiantsAthletics' : 'Giants Athletics',
                                            'CubsWhite Sox' : 'Cubs White-Sox',
                                            'Red Sox' : 'Red-Sox',
                                            'Blue Jays' : 'Blue-Jays'
    })

    cities['MLB'] = cities['MLB'].apply(lambda x: x.split(" "))
    cities = cities.explode('MLB')

    df = pd.merge(cities , mlb_df , left_on='MLB', right_on= 'team')
    
    df.drop(['team', 'MLB'], axis = 1 , inplace= True)
    df['W-L%'] = df['W-L%'].astype(float)
    
    df1 = df.groupby(['Metropolitan area']).mean()

    df2 = pd.merge( cities , df1 , on= 'Metropolitan area')

    df3 = df2.drop_duplicates('Metropolitan area')

    df3 = df3.filter(['Metropolitan area', 'Population (2016 est.)[8]', 'W-L%' ])


    return df3


def mlb_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = clean_data_mlb()

    population_by_region = df["Population (2016 est.)[8]"].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result

mlb_correlation()



(0.15003737475409493, 0.46442827201123427)

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [289]:
from distutils.command.clean import clean
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def clean_data_nfl():
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    
    nfl_df = nfl_df.loc[nfl_df['year']== 2018]

    nfl_df = nfl_df.filter(['team', 'W-L%'])

    nfl_df.drop([0, 5, 10, 15,20, 25,30,35] ,axis=0 , inplace=True)

    nfl_df['team'] = nfl_df['team'].apply(lambda x: re.sub(r'\+?\*?', "", x))

    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1])

    cities = cities.filter(['Metropolitan area','Population (2016 est.)[8]', 'NFL'])

    cities['NFL'] = cities['NFL'].apply(lambda x: re.sub(r'\[.*\]', '', x))

    cities['NFL'] = cities['NFL'].replace({
                                            'GiantsJets' : 'Giants Jets',
                                            'RamsChargers' : 'Rams Chargers',
                                            '49ersRaiders' : '49ers Raiders'
    })
    cities['NFL'] = cities['NFL'].apply(lambda x: x.split(' '))
    cities = cities.explode('NFL')

    df = pd.merge(cities , nfl_df , left_on='NFL', right_on='team')

    df = df.filter(['Metropolitan area', 'Population (2016 est.)[8]', 'W-L%'])

    df['W-L%'] = df['W-L%'].astype(float)

    df1 = df.groupby('Metropolitan area').mean()

    df2 = df.drop_duplicates('Metropolitan area').filter(['Metropolitan area', 'Population (2016 est.)[8]'])
    

    df3 = pd.merge(df2 , df1, on= 'Metropolitan area')

    return df3
    
def nfl_correlation(): 
    # YOUR CODE HERE
#     raise NotImplementedError()
    df = clean_data_nfl()
    
    population_by_region = df["Population (2016 est.)[8]"].astype(float) # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result
nfl_correlation()


(0.004282141436393019, 0.9824114740736553)

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [416]:
from heapq import merge
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def sports_team_performance():
    from scipy.stats import ttest_ind
    from scipy.stats import ttest_rel


    
    df1 = clean_data_nfl()
    df2 = clean_data_nba()
    df3= clean_data_mlb()
    df4 = clean_data_nhl()

    df5 = pd.merge(df1,df2, on='Metropolitan area').filter(['Metropolitan area', 'W-L%', 'W/L%'])
    df6 = pd.merge(df1,df3, on='Metropolitan area').filter(['Metropolitan area', 'W-L%_x', 'W-L%_y'])
    df7 = pd.merge(df1,df4, on='Metropolitan area').filter(['Metropolitan area', 'W-L%', 'W/L%'])
    df8 = pd.merge(df2,df3, on='Metropolitan area').filter(['Metropolitan area', 'W-L%', 'W/L%'])
    df9 = pd.merge(df2,df4, on='Metropolitan area').filter(['Metropolitan area', 'W/L%_x', 'W/L%_y'])
    df10 = pd.merge(df3,df4, on='Metropolitan area').filter(['Metropolitan area', 'W-L%', 'W/L%'])
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    p_values.loc[sports[0], sports[3]] = p_values.loc[sports[3], sports[0]]  = ttest_rel(df6['W-L%_x'], df6['W-L%_y'])[1] 
    p_values.loc[sports[0], sports[1]] = p_values.loc[sports[1], sports[0]] = ttest_rel(df5['W-L%'], df5['W/L%'])[1]
    p_values.loc[sports[0], sports[2]] = p_values.loc[sports[2], sports[0]] = ttest_rel(df7['W-L%'], df7['W/L%'])[1]
   
    p_values.loc[sports[1], sports[2]] = p_values.loc[sports[2], sports[1]]  = ttest_rel(df9['W/L%_x'], df9['W/L%_y'])[1] 
    p_values.loc[sports[1], sports[3]] = p_values.loc[sports[3], sports[1]] = ttest_rel(df8['W-L%'], df8['W/L%'])[1]
    
    p_values.loc[sports[2], sports[3]] = p_values.loc[sports[3], sports[2]] = ttest_rel(df10['W-L%'], df10['W/L%'])[1]
 
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
